In [29]:
print("Hello world")

Hello world


This .ipynb is an attempt at automating the checking of our steady states. 
The conclusions here are based off the work I have on my table on trying
to solve for equilibria and characteristic equations for matrices. 

In [30]:
import math
import numpy as np

In [31]:
"""
A couple nice functions that report steady states and their equilibrium. 
"""
def report_steady_state(S: float, L: float, G: float):
    print(f"Sparrow population: {S:.3f}")
    print(f"Locust population: {L:.3f}")
    print(f"Grain: {G:.3f}")

def report_equilibrium(*eigenvalues: float):
    """
    Prints eigenvalues and classifies stability. 

    TODO: Due to floating point, an == 0 check is unlikely. We may consider
    having some small value epsilon, and use this to check if things are zero. 
    """
    for i, lam in enumerate(eigenvalues, start=1):
        print(f"λ_{i} = {lam:.3f}")

    has_pos = any(lam > 0 for lam in eigenvalues)
    has_neg = any(lam < 0 for lam in eigenvalues)
    has_zero = any(lam == 0 for lam in eigenvalues)

    if has_pos:
        status = "unstable equilibrium"
    elif has_zero:
        status = "inconclusive"
    else: 
        status = "asymptotically stable equilibrium"

    print(f"Classification: {status}")


In [32]:
"""
Input parameters
"""
a1 = 1.0
a2 = 1
a3 = 1 / 3 
a4 = 1
Eh = 0.5

b1 = 1 
b2 = 0.15 * b1 
b3 = 0.01
b4 = 1.0
b5 = 1

c1 = 0.5
c2 = 0.5
c3 = 1.0

def f(L: float) -> float: 
    return b1 + (b2 * (L**2))/(b3**2 + L**2)

def dfdx(L: float) -> float:
    return (2 * b2 * (b3**2))/((b3**2 + L**2) ** 2)

Case A: Extinction Steady State
We set S and L equal to 0. 

In [33]:
"""
It is straightforwards to solve for the quantities at this equilibrium. 
"""

S_ext = 0
L_ext = 0
G_ext = 1/c3

report_steady_state(S_ext, L_ext, G_ext)

Sparrow population: 0.000
Locust population: 0.000
Grain: 1.000


In [34]:
"""
The Jacobian is a diagonal matrix. 
"""
l1_extinction = (a2/a3) - a3 - Eh 
l2_extinction = b1 - b4 
l3_extinction = -c3 

report_equilibrium(l1_extinction, l2_extinction, l3_extinction)

λ_1 = 2.167
λ_2 = 0.000
λ_3 = -1.000
Classification: unstable equilibrium


Case B: S = 0, L > 0

In [35]:
"""
Steady state populations when sparrows are extinct. 
"""
S_no_sparrows = 0 
L_no_sparrows = b3 * (math.sqrt((-1 * b1 + b4)/(b1 + b2 - b4)))
G_no_sparrows = 1 / ((c2 * L_no_sparrows) + c3)

report_steady_state(S_no_sparrows, L_no_sparrows, G_no_sparrows)

Sparrow population: 0.000
Locust population: 0.000
Grain: 1.000


In [36]:
"""
Eigenvalues when sparrows are extinct. 
"""
l1_no_sparrows = (a1 * L_no_sparrows + a2 * G_no_sparrows - a3 - Eh)
l2_no_sparrows = (dfdx(L_no_sparrows)) * L_no_sparrows
l3_no_sparrows = -1 / G_no_sparrows

report_equilibrium(l1_no_sparrows, l2_no_sparrows, l3_no_sparrows)

λ_1 = 0.167
λ_2 = 0.000
λ_3 = -1.000
Classification: unstable equilibrium


Case C: L = 0, S,G > 0

In [37]:
"""
Steady state populations when locusts are extinct. 
"""
S_C = (1 / c1) * (a2 / (a3 + Eh) - c3)
L_C = 0 
G_C = (a3 + Eh) / a2 

report_steady_state(S_C, L_C, G_C)


Sparrow population: 0.400
Locust population: 0.000
Grain: 0.833


In [38]:
"""
Eigenvalues when locusts are extinct. 
"""
lambda1_C = b1 - b4 - b5 * S_C

T = a2 / (a3 + Eh)
D = a2 - c3 * (a3 + Eh)

discriminant = T**2 - 4 * D

if discriminant < 0:
    raise ValueError(
        f"Complex solutions"
    )

sqrt_disc = discriminant**0.5

lambda2_C = (-T + sqrt_disc) / 2
lambda3_C = (-T - sqrt_disc) / 2

report_equilibrium(lambda1_C, lambda2_C, lambda3_C)

λ_1 = -0.400
λ_2 = -0.160
λ_3 = -1.040
Classification: asymptotically stable equilibrium


Case D: Coexistence. 

In [39]:
def coexistence_equation_scalar(L: float) -> float: 
    """
    It is possible to solve for the values of S and G in terms of L. 

    We build an equation whose root corresponds to the L at a steady state, 
    so we can calculate S and G from that root. 

    Deriving this function: 

    dL/dt = 0 gets us to f(L) - b4 - b5 * S = 0, so S(L) = (f(L) - b4) / b5 

    dG/dt = 0 gets us to G(L) = 1 / (c1 * S(L) + c2 * L + c3)

    dS/dt = 0 gets us to a1 * S * L + a2 * S * G - a3 * S - Eh * S = 0. 
    Since S > 0, we can write this as a1 * L + a2 * G = a3 + Eh. 

    > a1 * L + a2 * G = a3 + Eh <  is true at equilibrium. 
    So then, we can set F(L) = a1*L + a2 / (c1*S(L) + c2*L + c3) - (a3 + Eh)
    and having this value equal to 0 gives us the L at equilibrium. 

    We then use a root solver on F. 
    """
    if L <= 0:
        return float("nan")

    S = (f(L) - b4) / b5
    if S < 0:
        return float("nan")

    denom = c1*S + c2*L + c3
    G = 1.0 / denom
    if G < 0: 
        return float("nan")

    return a1*L + a2*G - (a3 + Eh)

In [40]:
from scipy.optimize import fsolve

def find_coexistence_equilibrium_fsolve():
    """
    Shove our problems at scipy and win. 
    """
    L0 = 1.0  


    L_star = fsolve(coexistence_equation_scalar, L0)[0]

    if L_star <= 0 or not np.isfinite(L_star):
        print("fsolve returned invalid L*")
        return None

    S_star = (f(L_star) - b4) / b5
    denom = c1*S_star + c2*L_star + c3
    G_star = 1.0 / denom

    if S_star < 0 or G_star < 0: 
        print("Invalid result for case D, coexistence. ")
        return None

    return S_star, L_star, G_star

In [41]:
"""
Stability at the coexistence steady state. 
"""
def jacobian_coexistence(S: float, L: float, G: float) -> np.ndarray:
    return np.array([
        [0.0,        a1 * S,          a2 * S],
        [-b5 * L,    dfdx(L) * L,     0.0   ],
        [-c1 * G,   -c2 * G,        -1.0 / G]
    ], dtype=float)

In [42]:
coexistence_result = find_coexistence_equilibrium_fsolve()
if coexistence_result:
    S_D, L_D, G_D = coexistence_result
    report_steady_state(S_D, L_D, G_D)

    J_star = jacobian_coexistence(S_D, L_D, G_D)
    eigvals = np.linalg.eigvals(J_star)

    report_equilibrium(*eigvals)
else:
    print("Could not find a steady state.")


Sparrow population: 0.150
Locust population: 1.000
Grain: 0.635
λ_1 = -0.024+0.351j
λ_2 = -0.024-0.351j
λ_3 = -1.526+0.000j
Classification: asymptotically stable equilibrium


/tmp/ipykernel_30079/963125692.py:10: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  L_star = fsolve(coexistence_equation_scalar, L0)[0]
